# Tensorflow Introduction

In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

2025-01-08 15:51:41.698541: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


TensorFlow version: 2.12.0


## Load Dataset
Load MNIST dataset for this part. The pixel values of the images range from 0 through 255. Scale these values to a range of 0 to 1 by dividing the values by `255.0`.

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

## Build the model

Layers are functions with a known mathematical structure that can be reused and have trainable variables.

Sequential is useful for stacking layers where each one has one input tensor and one output tensor.

Our model will use Flatten, Dense and Dropout layers

In [3]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

2025-01-08 15:53:34.997925: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-01-08 15:53:34.997990: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.23049322,  0.8898455 ,  0.5792415 , -0.15812199, -0.5561831 ,
        -0.20157102, -0.17321295, -0.2948543 ,  0.4478394 ,  0.01811334]],
      dtype=float32)

Converts logits to probabilities for each class

In [5]:
tf.nn.softmax(predictions).numpy()

array([[0.06949715, 0.21307081, 0.15618177, 0.07471322, 0.05017896,
        0.07153651, 0.0735942 , 0.06516514, 0.13695034, 0.08911186]],
      dtype=float32)

Define loss function

This takes a vector of ground truth values and a vector of logits and returns a scalar loss for each example. This loss is equal to the negative log probability of the true class. The value of the loss is zero of the model is sure of the correct class.

In [6]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [7]:
loss_fn(y_train[:1], predictions).numpy()

2.6375473

Configure and compile the model specifying optimizer, loss function and metrics

In [8]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## Train the model

This takes a few minutes depending on system configuration

In [9]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5


2025-01-08 16:03:51.057806: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-08 16:03:51.095854: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


1875/1875 [==============================] - 17s 9ms/step - loss: 0.4031 - accuracy: 0.8844
Epoch 2/5
1875/1875 [==============================] - 16s 9ms/step - loss: 0.3657 - accuracy: 0.8995
Epoch 3/5
1875/1875 [==============================] - 16s 9ms/step - loss: 0.3772 - accuracy: 0.8988
Epoch 4/5
1875/1875 [==============================] - 16s 9ms/step - loss: 0.3918 - accuracy: 0.8951
Epoch 5/5
1875/1875 [==============================] - 16s 9ms/step - loss: 0.4023 - accuracy: 0.8936


## Evaluate the model

In [10]:
model.evaluate(x_test,  y_test, verbose=2)

2025-01-08 16:05:55.574971: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


313/313 - 2s - loss: 0.3322 - accuracy: 0.9135 - 2s/epoch - 6ms/step


[0.3322332799434662, 0.9135000109672546]

In [11]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [12]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[8.93009652e-04, 5.34906377e-08, 2.85597565e-03, 2.89007518e-02,
        3.89518646e-05, 1.39928213e-03, 3.53052371e-07, 9.65741932e-01,
        7.46102232e-05, 9.51473121e-05],
       [1.60889863e-03, 2.63321155e-04, 8.50430846e-01, 1.28267273e-01,
        3.29809836e-07, 1.20081371e-02, 2.86986562e-03, 8.99159236e-09,
        4.55081882e-03, 5.49024264e-07],
       [1.51621780e-05, 7.88244009e-01, 5.00864599e-06, 2.12785471e-06,
        1.17641808e-02, 3.28442460e-04, 5.23880939e-04, 1.96060359e-01,
        8.51460754e-06, 3.04826419e-03],
       [3.26923609e-01, 6.64071238e-04, 7.80879236e-06, 1.35804953e-06,
        3.33907418e-02, 2.99694104e-04, 3.23114358e-02, 6.05931401e-01,
        5.66980532e-07, 4.69295395e-04],
       [2.67761578e-07, 4.17045537e-10, 2.20935936e-10, 2.33003616e-09,
        9.80020940e-01, 1.43483685e-05, 2.34527179e-05, 7.64845812e-04,
        3.89091355e-07, 1.91757102e-02]], dtype=float32)>